In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
model_path = "/content/drive/My Drive/ML project 2/levin/trained_models/segformer_raw.pt"

In [3]:
import sys
sys.path.append('/content/drive/My Drive/ML project 2/levin')

In [4]:
import torch
import os
from SegFormer.segformer import SegFormer
from PIL import Image
import numpy as np
import matplotlib.image as mpimg
import shutil

In [5]:
model = SegFormer()

# Load the saved state dictionary
model.model.load_state_dict(torch.load(model_path, map_location=model.device))
model.model.to(model.device)

print("Model loaded successfully!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([1, 256, 1, 1]) in the model instantiated
You sho

Model loaded successfully!


In [6]:
# Get paths for test images
def get_test_images(test_folder):
    """
    Recursively retrieve all test image file paths from the test folder.
    :param test_folder: Path to the folder containing test images in subfolders.
    :return: List of file paths to test images.
    """
    image_paths = []
    for root, _, files in os.walk(test_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(root, file))
    return image_paths

In [16]:
paths = get_test_images("/content/drive/My Drive/ML project 2/data/test_set_images")
paths.sort(key=lambda x: int(os.path.basename(x).split('_')[1].split('.')[0]))
paths

['/content/drive/My Drive/ML project 2/data/test_set_images/test_1/test_1.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_2/test_2.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_3/test_3.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_4/test_4.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_5/test_5.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_6/test_6.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_7/test_7.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_8/test_8.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_9/test_9.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_10/test_10.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_11/test_11.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_12/test_12.png',
 '/content/drive/My Drive/ML proje

In [7]:
def save_predictions_as_images(image_paths, model, output_folder):
    """
    Save predicted masks as images in the output folder.
    :param image_paths: List of test image file paths.
    :param model: SegFormer model for prediction.
    :param output_folder: Path to save predicted masks.
    """
    os.makedirs(output_folder, exist_ok=True)

    for idx, image_path in enumerate(image_paths, start=1):
        # Load and predict mask
        image = Image.open(image_path).convert("RGB")
        predicted_mask = model.predict(image)  # Shape: [height, width]

        # Save mask as an image
        output_path = os.path.join(output_folder, f"mask_{idx:03d}.png")
        Image.fromarray((predicted_mask * 255).astype(np.uint8)).save(output_path)

    print(f"Predicted masks saved to {output_folder}")

In [10]:
foreground_threshold = 0.25  # Threshold for patch classification

# Assign a label to a patch based on the foreground threshold
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0

# Generate submission strings from a single mask image
def mask_to_submission_strings(image_filename, img_number):
    """
    Reads a single mask image and outputs the strings for the submission file.
    :param image_filename: Path to the mask image file.
    :param img_number: Image number (used in the submission ID).
    :yield: Submission strings for each 16x16 patch.
    """
    im = mpimg.imread(image_filename)
    print(f"Image size: {im.shape}")
    patch_size = 16
    height, width = im.shape[:2]

    # Iterate over the full image resolution
    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            patch = im[y:y + patch_size, x:x + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},".format(img_number, y, x) + str(label))

# Convert all masks into a single submission file
def masks_to_submission(submission_filename, image_filenames):
    """
    Converts predicted masks into a submission file.
    :param submission_filename: Path to save the submission file.
    :param image_filenames: List of mask image file paths.
    """
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')  # Write header
        for img_number, image_filename in enumerate(image_filenames, start=1):
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(image_filename, img_number))

In [13]:
def preprocess_test_image(image_path, target_size=(512, 512)):
    """
    Preprocess a test image to match the model's input size.
    """
    image = Image.open(image_path).convert("RGB")
    image = image.resize(target_size, Image.BILINEAR)
    return image


def postprocess_prediction(predicted_mask, original_size=(608, 608)):
    """
    Resize the model's predicted mask back to the original image size.
    """
    from PIL import Image
    import numpy as np

    mask_image = Image.fromarray((predicted_mask * 255).astype(np.uint8))  # Convert to image for resizing
    mask_image = mask_image.resize(original_size, Image.NEAREST)  # Resize back to 608x608
    return np.array(mask_image) // 255  # Convert back to binary mask

In [14]:
# Define paths
test_images_folder = "/content/drive/My Drive/ML project 2/data/test_set_images"
output_masks_folder = "/content/drive/My Drive/ML project 2/levin/predictions"
submission_file = "/content/drive/My Drive/ML project 2/levin/segformer_raw_submission.csv"

# Clear predictions folder
if os.path.exists(output_masks_folder):
    shutil.rmtree(output_masks_folder)
os.makedirs(output_masks_folder, exist_ok=True)

# Retrieve and sort test image paths (simplified sorting)
test_image_paths = get_test_images(test_images_folder)
test_image_paths.sort(key=lambda x: int(os.path.basename(x).split('_')[1].split('.')[0]))

# Process each test image
for idx, image_path in enumerate(test_image_paths, start=1):
    test_image = preprocess_test_image(image_path)  # Resize to 512x512
    predicted_mask = model.predict(test_image)      # Predict (512x512)
    resized_mask = postprocess_prediction(predicted_mask, original_size=(608, 608))  # Resize back to 608x608

    # Save resized mask
    mask_path = os.path.join(output_masks_folder, f"mask_{idx:03d}.png")
    Image.fromarray((resized_mask * 255).astype(np.uint8)).save(mask_path)

# Generate submission file
predicted_mask_paths = sorted([os.path.join(output_masks_folder, f) for f in os.listdir(output_masks_folder)], key=lambda x: int(os.path.basename(x).split('_')[1].split('.')[0]))
masks_to_submission(submission_file, predicted_mask_paths)

print(f"Submission file created at {submission_file}")

# Validate row count
with open(submission_file, "r") as f:
    rows = f.readlines()
print(f"Total rows (excluding header): {len(rows) - 1}")

Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: (608, 608)
Image size: